In [ ]:
"""
    관련Library 활용 : torchivision.transforms등의이미지데이터를변형하는다양한방법을이해하고있나
"""
    

In [ ]:
""" 요약 """
grid = torchvision.utils.make_grid(images_dn, nrow=8)  # 그리드로 묶기(C,H,W)
plt.imshow(grid.permute(1,2,0))  # CHW -> HWC 로 변환 후 표시



In [ ]:
def denorm(x): # -2.5~2.5
    mean = torch.tensor(CIFAR10_MEAN).view(1,3,1,1)  # (1,3,1,1) 형태로 broadcast
    std  = torch.tensor(CIFAR10_STD).view(1,3,1,1)  # (1,3,1,1) 형태로 broadcast
    return x * std + mean  # 정규화 역변환, 0~1

images, labels = next(iter(train_loader))  # 한 배치 가져오기
images_dn = denorm(images[:16]).clamp(0,1)  # 앞 16장만 역정규화 후 [0,1]로 clip

grid = torchvision.utils.make_grid(images_dn, nrow=8)  # 그리드로 묶기(C,H,W)
plt.figure(figsize=(12,4))
plt.imshow(grid.permute(1,2,0))  # CHW -> HWC 로 변환 후 표시
plt.axis('off')
plt.title('CIFAR-10 samples (denormalized)')
plt.show()

print('labels:', [class_names[int(x)] for x in labels[:16]])  # 라벨 이름 출

In [1]:
# Attention의 einsum도 vision library라고 할 수 있지 않을까? 
q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), qkv)  # multi-head 연산을 위해 각 head 기준 n x d 로 분리
# 아래 두 코드를 합친게 위 한줄 인 듯. h만 주면 나머지 d는 계산을 해주는...
#  keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
#  keys = keys.transpose(1, 2)

dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale  # 각 토큰 간 유사도(Q·Kᵀ) 계산 후 스케일 적용

attn = self.attend(dots)  # 토큰 간 가중치(attention map) 생성
self.last_attn = attn.detach()  # 그래프에서 분리(detach)해서 저장 (시각화 목적)

out = einsum('b h i j, b h j d -> b h i d', attn, v)  # attention 가중합으로 새로운 토큰 표현(out) 계산
out = rearrange(out, 'b h n d -> b n (h d)')  # 헤드 차원을 다시 합쳐 (batch, tokens, dim)로 복원

In [ ]:
def show_batch(data_loader, max_images: int = 16, nrow: int = 4, figsize=(10, 10)):
    """배치에서 일부 샘플을 보기 좋게 시각화합니다.

    - 기본값은 16장(4×4)을 비교적 크게 보여주는 설정입니다.
    - max_images/nrow/figsize를 바꾸면 '더 적게/더 많이', '더 크게/더 작게'를 조절할 수 있습니다.

    - CIFAR-10은 (C,H,W)=(3,32,32) RGB이므로, 시각화할 때는 (H,W,C)로 바꿔야 합니다.
    - make_grid 결과가 figure 안에서 '밀려 보이는' 경우가 있어, subplot 여백을 0으로 맞춰 꽉 차게 표시합니다.

    Args:
        data_loader: DataLoader
        max_images: 한 번에 보여줄 이미지 개수(기본 16장)
        nrow: grid 한 줄에 배치할 이미지 개수(기본 4개)
        figsize: figure 크기
    """
    batch = next(iter(data_loader))
    images, labels = batch  # images: (B,C,H,W)
    images = images[:max_images]

    # make_grid는 (B,C,H,W) → grid(C,H,W)
    grid = torchvision.utils.make_grid(images, nrow=nrow, padding=2)

    # 시각화는 정규화 해제 후 HWC로 변환
    grid = denormalize_cifar10(grid).detach().cpu().numpy().transpose((1, 2, 0))

    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(grid, interpolation="nearest")
    ax.set_title(f"Batch Samples (CIFAR-10) | shown={len(images)}")
    ax.axis("off")

    # 여백 제거: grid가 4×4에 꽉 차도록
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.show()

# 미리보기: 크게(figure) + 적게(16장) 보기
show_batch(train_loader, max_images=16, nrow=4, figsize=(10, 10))
show_batch(train_loader, max_images=16, nrow=4, figsize=(10, 10))